# 連線用

In [3]:
import pandas as pd
import pymongo
myclient = pymongo.MongoClient("localhost",27017)
mydb = myclient["CDC"]
namelist = mydb.list_collection_names()
print(namelist)

# 讀取檔案 改檔名就好

In [4]:
room = pd.read_csv('input.csv')
print(room)

# 確認機器位置

In [5]:
device = list(room['device'])
print(device)

where = list(room['where'])
print(where)

# 檢查機器位址，記得醫院地點要改

In [6]:
temp = []

loc = 'loc'
for i in range(len(room)):
    temp.append({'name':room.iloc[i]['where'],'id':room.iloc[i]['device'],'loc':loc,'status':'online'})

print(temp)

# 加進mongo

In [32]:
for i in range(len(temp)):
    mydb['device'].insert_one(temp[i])

# 檢查db有無 改地址

In [7]:
depart = mydb['device'].find({'loc':loc})
for i in depart:
    print(i)

# 增加新醫院地點 lon lat token loc num name _id

In [8]:
lon = 00.0000000
lat = 00.0000000
token = 'token'
name = 'name'
temp1 = ({"_id":"020",'loc':loc,'token':token,'num':17,'name':name,'lat':lat,'lon':lon})
print(temp1)

# 確認完成後再加db

In [9]:
mydb['token'].insert_one(temp1)

# 刪除用

In [10]:
#mydb['token'].delete_one({'loc':'loc'})
c = mydb['token'].find()
for i in c:
    print(i)

In [11]:
data = mydb['device'].find({'loc':'loc'})
for i in data:
    print(i)

# 更新單筆資料

In [12]:
sub = mydb["subscribe"].find()
for i in sub:
    print(i)

In [13]:
query = {'_id':'016'}
new = {'$set':{'name': 'name'}}
mydb['token'].update_one(query,new)

# 更新多筆資料

In [ ]:
auth = ([d for d in mydb['device'].find()])
for i in auth:
    query = {'loc':'loc'}
    new = {'$set':{'loc':'loc'}}
    mydb['device'].update_one(query,new)

# 更新db的上下線

In [192]:
import requests
import numpy as np
from bs4 import BeautifulSoup as BS
import pymongo
myclient = pymongo.MongoClient("localhost",27017)
mydb = myclient["CDC"]
url = "https://pm25.lass-net.org/IAQ/detail.php?city=iaq-cdc"
res = requests.get(url)
data = BS(res.text,'html.parser')
target = data.find_all('tr')
for i in target:
    a = i.find_all('td')
    if len(a[1].text)>2:
        id_ = a[2].text
        status = i.find_all('img')[0].get('alt')
#         print(id_,status)
        myquery = {'id':id_}
        newvalues = {'$set':{'status':status}}
        mydb['device'].update_one(myquery,newvalues)
'''
MAPS
'''
url = 'https://pm25.lass-net.org/AirBox/detail.php?city=maps6-edu'
r = requests.get(url)
data = BS(r.text,'html.parser')
target = data.find_all('tr')
for j,i in enumerate(target):
    a = i.find_all('td')
    if a:
        status = 'offline' if a[1].find_all('svg')[0].get('fill')=='#000000' else 'online'
        id_ = a[2].text.replace('\n','').replace(' ','')
        if 'NTNU' in a[1].text:
#             print(id_,status)
            myquery = {'id':id_}
            newvalues = {'$set':{'status':status}}
            mydb['device'].update_one(myquery,newvalues)
